In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import PIL
import pytesseract
import os
import requests
import urllib.parse
from PIL import Image

In [38]:

def capture_screenshot(url, width, height, output_path):
    """
    Function to capture a screenshot of a webpage using mini.s-shot.ru.
    """
    BASE = f'https://mini.s-shot.ru/{width}x{height}/JPEG/1280/Z100/?'
    encoded_url = urllib.parse.quote_plus(url)
    full_url = BASE + encoded_url
    
    response = requests.get(full_url, stream=True)
    if response.status_code == 200:
        with open(output_path, 'wb') as file:
            for chunk in response:
                file.write(chunk)
        print(f"Screenshot saved as {output_path}")
    else:
        print(f"Failed to capture screenshot. Status code: {response.status_code}")

def crop_table_from_image(image_path, output_path, crop_box):
    """
    Function to crop the table region from an image.
    """
    # Load the image
    img = Image.open(image_path)
    
    # Crop the image using the coordinates (left, upper, right, lower)
    cropped_img = img.crop(crop_box)
    
    # Save the cropped image
    cropped_img.save(output_path)
    
    print(f"Cropped table saved as {output_path}")

# Step 1: Capture the screenshot
url = 'https://www.cmegroup.com/markets/equities/sp/e-mini-sandp500.volume.html'
screenshot_path = 'cme_screenshot_large.jpg'

# Screenshot size (you can adjust these dimensions if necessary)
capture_screenshot(url, 1280, 4000, screenshot_path)

# Step 2: Crop the table from the screenshot
crop_box = (20, 1350, 1280, 1800)  # Example coordinates to crop the table from the screenshot
output_cropped_table = 'cropped_table_image.png'
crop_table_from_image(screenshot_path, output_cropped_table, crop_box)

Screenshot saved as cme_screenshot_large.jpg
Cropped table saved as cropped_table_image.png


In [39]:
print(pytesseract.get_languages())
special_config = '--psm 12 --oem 1'
languages_ = "eng"

['eng', 'osd']


In [40]:
image_path = "cropped_table_image.png" # Any image format will do
img_pl=PIL.Image.open(image_path) # You can use opencv for that option too

In [41]:

# Extract text from image using pytesseract
data = pytesseract.image_to_string(img_pl)

# Split the extracted text into lines for further processing
lines = data.split("\n")

# Example parsing logic to extract relevant rows (for illustration purposes)
# We're assuming 'lines' contains the extracted text row-wise from the image.
# You can inspect and clean this data accordingly.
data = []
for line in lines:
    # Look for lines with "SEP 2024" to extract relevant information
    if "DEC 2024" in line:
        # Split the line into columns and extract "TOTAL VOLUME" and "AT CLOSE" (assuming fixed positions)
        # Adjust the indices based on the actual output from pytesseract
        columns = line.split()
        month = columns[0]
        total_volume = columns[4]  # Adjust based on actual index from the extracted text
        at_close = columns[-2]     # Adjust based on actual index from the extracted text
        data.append([month, total_volume, at_close])

# Convert the extracted data into a pandas DataFrame
df = pd.DataFrame(data, columns=['Month', 'Total Volume', 'At Close'])

# Display the DataFrame
df


,Month,Total Volume,At Close
0,DEC,"1,250,353","2,066,996"
